In [13]:
from gurobipy import Model, GRB, quicksum

Defining Teams and Strength Categories

In [14]:
num_teams = 10
teams = list(range(num_teams))

In [15]:
num_weeks = num_teams-1
weeks = list(range(num_weeks))

In [16]:
# Dictionary of Ranking of Teams
# 
ranking = {
    1: teams[:3],  # Category 1
    2: teams[3:6], # Category 2
    3: teams[6:],  # Category 3
}

In [17]:
def get_stronger_teams(team: int, ranking: dict) -> list:
    if team in ranking[3]:
        return list(ranking[1]) + list(ranking[2])
    elif team in ranking[2]:
        return list(ranking[1])
    elif team in ranking[1]:
        return []

In [18]:
# Set of teams located in the same town
D = [(0,5)]  

Defining Variables

In [19]:
# Create the model
model = Model("Round_Robin_Scheduling")

# Binary decision variable: x[i,j,w] is 1 if team i is the home team and team j is the away team on week w
x = model.addVars(num_teams, num_teams, num_weeks, vtype=GRB.BINARY, name="x")

# Binary decision variable: y[i,w] is 1 if team i is playing at home on week w and w+1
y = model.addVars(num_teams, num_weeks-1, vtype=GRB.BINARY, name="y")

# Binary decision variable: z[i,w] is 1 if team i is playing at away on week w and w+1
z = model.addVars(num_teams, num_weeks-1, vtype=GRB.BINARY, name="z")

# Binary variable: s[i,w] is 1 if consecutive unfair matchups occur on week w and w+1
s = model.addVars(num_teams, num_weeks-1, vtype=GRB.BINARY, name="s ")

Objective function 

In [20]:
# Objective function (4.1a)
model.setObjective(
    quicksum(s[i,w] for i in teams for w in weeks[:-1]),
    GRB.MINIMIZE
    )

Format Requirements for a SRR schedule

In [21]:
# Constraint (4.1b): Each team plays against every other team exactly once                    
for i in teams:
    for j in teams:
        if i < j:
            model.addConstr(quicksum(x[i, j, w] + x[j, i, w] for w in weeks) == 1, name="4.1b_{}_{}".format(i, j))

# Constraint (4.1c): Each team plays exactly once per week
for i in teams:
    for w in weeks:
        model.addConstr(quicksum(x[i, j, w] + x[j, i, w] for j in teams if j != i) == 1, name="4.1c_{}_{}".format(i, w))

# Constraint (4.1e): Home/away balance for each team
for i in teams:
    model.addConstr(quicksum(x[i, j, w] for j in teams if j != i for w in weeks) >=  num_weeks // 2     , name="4.1e_lwr_{}".format(i))
for i in range(num_teams):
    model.addConstr(quicksum(x[i, j, w] for j in teams if j != i for w in weeks) <= (num_weeks // 2) + 1, name="4.1e_upr_{}".format(i))

Breaks Requirement

In [22]:
# Preventing consecutive Home Breaks
# Constraint (4.1h)
for i in teams:
    for w in weeks[:-1]:
        for j in teams:
            for j_prime in teams:
                if  i != j and j != j_prime and i != j_prime:
                    model.addConstr(x[i, j, w] + x[i, j_prime, w+1] <= 1 + y[i, w], name="4.1h_{}_{}_{}".format(i, j, j_prime))
# Constraint (4.1l) 
for i in teams:
    for w in weeks[:-2]:
        model.addConstr(y[i, w] + y[i, w+1] <= 1, name="4.1l_{}_{}".format(i, w))


# Prevent consecutive Away Breaks
# Constraint (4.1k)
for i in teams:
    for w in weeks[:-1]:
        for j in teams:
            for j_prime in teams:
                if  i != j and j != j_prime and i != j_prime:
                    model.addConstr(x[j, i, w] + x[j_prime, i, w+1] <= 1 + z[i, w], name="4.1k_{}_{}_{}".format(i, j, j_prime))
# Constraint (4.1m) 
for i in teams:
    for w in weeks[:-2]:
        model.addConstr(z[i, w] + z[i, w+1] <= 1, name="4.1m_{}_{}".format(i, w))

Venue Restriction

In [23]:
# Constraint: Cities (Stadium) Restriction                              
for (ii,jj) in D:
    for w in weeks:
        model.addConstr(   quicksum(x[ii,j,w] for j in teams if j!=ii)
                         + quicksum(x[jj,i,w] for i in teams if i!=jj) == 1, name="4.1n_{}_{}_{}".format(ii,jj,w))

Strength Constraints

In [24]:
# Constraint (4.1o)
for i in teams:
  S_i = get_stronger_teams(i, ranking)
  for w in weeks[:-1]:
    model.addConstr(s[i,w] >= quicksum(x[i,j,w]   + x[j,i,w]   for j in S_i)
                                    + 
                              quicksum(x[i,j,w+1] + x[j,i,w+1] for j in S_i) 
                                    - 1, name="4.1o_{}_{}".format(i, w))

# Constraint (4.1p)
for i in teams:
    S_i = get_stronger_teams(i, ranking)
    for w in weeks[:-1]:
      model.addConstr(s[i,w] <= quicksum(x[i,j,w]   + x[j,i,w]   for j in S_i), name="4.1p_{}_{}".format(i, w))

# Constraint (4.1q)
for i in teams:
    S_i = get_stronger_teams(i, ranking)
    for w in weeks[:-1]:
      model.addConstr(s[i,w] <= quicksum(x[i,j,w+1] + x[j,i,w+1] for j in S_i), name="4.1q_{}_{}".format(i, w))

Solving the Model

In [ ]:
%%time

# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12064 rows, 1140 columns and 41404 nonzeros
Model fingerprint: 0x4180fcdb
Variable types: 0 continuous, 1140 integer (1140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 112 rows and 154 columns
Presolve time: 0.07s
Presolved: 11952 rows, 986 columns, 41252 nonzeros
Variable types: 0 continuous, 986 integer (986 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 8.000000e+00, 398 iterations, 0.05 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

In [26]:
print(f"Total Constraints: {model.NumConstrs}")
print(f"Total Variables:   {model.NumVars}"   )

Total Constraints: 12064
Total Variables:   1140


Displaying Results

In [27]:
# Output the schedule if feasible is found
if model.status == GRB.OPTIMAL:
    print("Optimal Schedule:")
    for w in weeks:
        print(f"Week {w + 1}:")
        for i in teams:
            for j in teams:
                if i != j and x[i, j, w].x > 0.5:
                    # Normal print
                    # print(f"  Team {i + 1} (Home) vs. Team {j + 1} (Away)")
                    
                    # Checking if schedule (without H/A specs) works
                    if i > j:
                        print(f"{j+1} v {i+1}:         Team {i + 1} (Home) vs. Team {j + 1} (Away)")
                    else:
                        print(f"{i+1} v {j+1}:         Team {i + 1} (Home) vs. Team {j + 1} (Away)")
                    
    print(f"\nTotal Cost (Optimized Objective Value): {model.objVal}")

else:
    print("No optimal solution found.")
    # model.status == GRB.INFEASIBLE:
    model.computeIIS()  # Compute Irreducible Inconsistent Subsystem (IIS)
    model.write("infeasible_constraints.ilp")  # Save infeasible constraints to file


Optimal Schedule:
Week 1:
2 v 9:         Team 2 (Home) vs. Team 9 (Away)
5 v 7:         Team 5 (Home) vs. Team 7 (Away)
4 v 6:         Team 6 (Home) vs. Team 4 (Away)
3 v 8:         Team 8 (Home) vs. Team 3 (Away)
1 v 10:         Team 10 (Home) vs. Team 1 (Away)
Week 2:
1 v 8:         Team 1 (Home) vs. Team 8 (Away)
2 v 5:         Team 2 (Home) vs. Team 5 (Away)
3 v 4:         Team 3 (Home) vs. Team 4 (Away)
7 v 10:         Team 7 (Home) vs. Team 10 (Away)
6 v 9:         Team 9 (Home) vs. Team 6 (Away)
Week 3:
1 v 3:         Team 3 (Home) vs. Team 1 (Away)
4 v 7:         Team 4 (Home) vs. Team 7 (Away)
5 v 10:         Team 5 (Home) vs. Team 10 (Away)
2 v 6:         Team 6 (Home) vs. Team 2 (Away)
8 v 9:         Team 8 (Home) vs. Team 9 (Away)
Week 4:
1 v 4:         Team 1 (Home) vs. Team 4 (Away)
5 v 8:         Team 5 (Home) vs. Team 8 (Away)
6 v 7:         Team 7 (Home) vs. Team 6 (Away)
3 v 9:         Team 9 (Home) vs. Team 3 (Away)
2 v 10:         Team 10 (Home) vs. Team 2 (Away)
We